In [21]:
# -*- coding: utf-8 -*-
"""
Created on 01/14, 2023
@author: WillyF

"""

# https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html
# https://opendata.cwb.gov.tw/dataset/forecast/F-C0032-001
# https://opendata.cwb.gov.tw/dist/opendata-swagger.html?urls.primaryName=openAPI#/%E9%A0%90%E5%A0%B1/get_v1_rest_datastore_F_C0032_001
# https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html
# https://stackoverflow.com/questions/31306741/unmelt-pandas-dataframe
# https://medium.com/%E6%95%B8%E6%93%9A%E4%B8%8D%E6%AD%A2-not-only-data/pandas-%E5%BF%AB%E9%80%9F%E7%9E%AD%E8%A7%A3-pivot-table-%E8%88%87%E6%87%89%E7%94%A8-21e4c37b9216
# https://docs.python.org/3/library/stdtypes.html#str.isdigit
# https://datagy.io/python-string-to-date/
# https://www.itsolutionstuff.com/post/how-to-check-if-today-is-wednesday-or-not-in-pythonexample.html
# https://stackoverflow.com/questions/19289171/importing-a-variable-from-one-python-script-to-another
# https://blog.netwrix.com/2022/11/14/how-to-hide-api-keys-github/



'\nCreated on 01/14, 2023\n@author: WillyF\n\n'

In [22]:
import pandas as pd
# import urllib
import urllib.request
import json
import numpy as np
# from glob import glob
import os
import warnings
from datetime import datetime
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)
from apikeyconfig import api_key_WF

In [23]:
TW_Region = pd.read_csv(
        "https://github.com/LeBronWilly/TW_Weather_FCST/raw/main/TW_Region.csv",
        encoding='utf8')
# TW_Region

In [24]:
data_source = "https://opendata.cwb.gov.tw/api/v1/rest/datastore/F-C0032-001?Authorization="+api_key_WF+"&format=JSON&locationName=&elementName=&sort=time"
json_url = urllib.request.urlopen(data_source)
data = json.loads(json_url.read())
# data

In [25]:
data_df = pd.json_normalize(data["records"],
                            record_path = ["location","weatherElement","time"],
                            meta = [['location',"locationName"],
                                    ['location',"weatherElement","elementName"],
                                    'datasetDescription',
                                   ],
                           )
data_df.columns = [x.split(".")[-1] for x in data_df.columns]
data_df["parameterUnit"].replace("百分比", "%", inplace=True)
data_df["parameterUnit"].replace("C", "°C", inplace=True)
data_df["parameterUnit"].replace(np.nan, "", inplace=True)
data_df["elementName"].replace("Wx", "Weather Forcast", inplace=True)
data_df["elementName"].replace("PoP", "Probability of Precipitation", inplace=True)
data_df["elementName"].replace("MinT", "Min Temperature", inplace=True)
data_df["elementName"].replace("MaxT", "Max Temperature", inplace=True)
data_df["elementName"].replace("CI", "Comfort Index", inplace=True)
data_df["Parameter"] = data_df["parameterName"] + data_df["parameterUnit"]
data_df["startTime"] = data_df["startTime"].apply(lambda x: x.replace("-", "/")[:-3])
data_df["endTime"] = data_df["endTime"].apply(lambda x: x.replace("-", "/")[:-3])
data_df = data_df.merge(TW_Region, how="left", left_on='locationName', right_on='City/County', validate="many_to_one")
data_df["locationName"].replace("臺", "台", inplace=True, regex=True)

# data_df

In [26]:
data_df_pivot = pd.pivot_table(data_df,
                               index=["startTime", "endTime", "Region", 'locationName'], 
                               columns=["elementName"], 
                               values=["Parameter"],
                               aggfunc= lambda x: x).reset_index()
data_df_pivot.columns = ["startTime","endTime", "Region","locationName","Comfort Index",
                         "Max Temperature","Min Temperature","Probability of Precipitation","Weather Forcast"]
data_df_pivot = data_df_pivot.sort_values(by=["locationName","startTime","endTime"]).reset_index(drop=True)
data_df_pivot["Period"] = data_df_pivot["startTime"].str.cat(data_df_pivot["endTime"], sep =" ~ ")
data_df_pivot["Temperature"] = data_df_pivot["Max Temperature"].str.cat(data_df_pivot["Min Temperature"], sep =" ~ ")
data_df_pivot = data_df_pivot.rename(columns={"locationName": "Location", "Probability of Precipitation": "PoP", "Weather Forcast": "Weather FCST"}, errors="raise")

# data_df_pivot

In [42]:
TW_Region = pd.read_csv(
        "https://github.com/LeBronWilly/TW_Weather_FCST/raw/main/TW_Region.csv",
        encoding='utf8')
# TW_Region

In [43]:
data_source = "https://opendata.cwb.gov.tw/api/v1/rest/datastore/F-D0047-091?Authorization="+api_key_WF+"&sort=time"
json_url = urllib.request.urlopen(data_source)
data = json.loads(json_url.read())
# data

In [44]:
days_name = ["Mon.","Tue.","Wed.","Thu.","Fri.","Sat.","Sun."]
data_df = pd.json_normalize(data["records"],
                            record_path = ['locations',"location","weatherElement",'time',"elementValue"],
                            meta = [['locations','location','locationName'],
                                    ['locations','location','lon'],
                                    ['locations','location','lat'],
                                    ['locations',"location","weatherElement","elementName"],
                                    ['locations',"location","weatherElement","description"],
                                    ['locations',"location","weatherElement",'time',"startTime"],
                                    ['locations',"location","weatherElement",'time',"endTime"]])
data_df.columns = ["Value","Unit","Location","Long","Lat","Element_EN","Element","StartTime","EndTime"]
data_df = data_df[["StartTime","EndTime","Location","Element","Element_EN","Value","Unit","Long","Lat"]]
data_df = data_df[data_df["Unit"]!="自定義 Wx 單位"].reset_index(drop=True)
data_df["Unit"].replace("百分比", "%", inplace=True)
data_df["Unit"].replace("8方位", "", inplace=True)
data_df["Unit"].replace("紫外線指數", "", inplace=True)
data_df["Unit"].replace("攝氏度", "°C", inplace=True)
data_df["Unit"].replace("公尺/秒", "m/s", inplace=True)
data_df["Unit"].replace("自定義 CI 文字", "", inplace=True)
data_df["Unit"].replace("自定義 Wx 文字", "", inplace=True)
data_df["Unit"].replace("自定義 Wx 單位", "", inplace=True)
data_df.loc[(data_df["Element"].str.contains("紫外線指數")) & (data_df["Unit"].str.contains("曝曬級數")),
            "Element"] = "曝曬級數"
data_df.loc[(data_df["Element"].str.contains("最大風速")) & (data_df["Unit"].str.contains("蒲福風級")),
            "Element"] = "蒲福風級"
data_df.loc[(data_df["Element"].str.contains("最大舒適度指數")) & (~data_df["Value"].str.isdigit()),
            "Element"] = "最大舒適度"
data_df.loc[(data_df["Element"].str.contains("最小舒適度指數")) & (~data_df["Value"].str.isdigit()),
            "Element"] = "最小舒適度"
data_df["Unit"].replace("曝曬級數", "", inplace=True)
data_df["Unit"].replace("蒲福風級", "", inplace=True)
data_df["Unit"].replace("NA", "", inplace=True)
data_df["Unit"].replace("NA ", "", inplace=True)
data_df["Unit"].replace(" ", "", inplace=True)
data_df["Value"].replace("<= 1", "≤1", inplace=True)
data_df["Parameter"] = data_df["Value"] + data_df["Unit"]
data_df["Parameter"].replace(" %", "-", inplace=True)
data_df["StartTime"] = data_df["StartTime"].apply(lambda x: x.replace("-", "/")[:-3])
data_df["StartTime"] = data_df["StartTime"].apply(lambda x: x+" ("+days_name[datetime.strptime(x, '%Y/%m/%d %H:%M').weekday()]+")")
data_df["Date"] = data_df["StartTime"].apply(lambda x: x.split(" ")[0]+" "+x.split(" ")[-1])
data_df["EndTime"] = data_df["EndTime"].apply(lambda x: x.replace("-", "/")[:-3])
data_df["EndTime"] = data_df["EndTime"].apply(lambda x: x+" ("+days_name[datetime.strptime(x, '%Y/%m/%d %H:%M').weekday()]+")")
data_df = data_df.merge(TW_Region, how="left", left_on='Location', right_on='City/County', validate="many_to_one")
data_df["Location"].replace("臺", "台", inplace=True, regex=True)
data_df["Long"] = data_df["Long"].astype(float)
data_df["Lat"] = data_df["Lat"].astype(float)
data_df

,StartTime,EndTime,Location,Element,Element_EN,Value,Unit,Long,Lat,Parameter,Date,City/County,Region
0,2023/08/06 12:00 (Sun.),2023/08/06 18:00 (Sun.),新竹縣,12小時降雨機率,PoP12h,10,%,120.995698,24.841245,10%,2023/08/06 (Sun.),新竹縣,北部地區
1,2023/08/06 18:00 (Sun.),2023/08/07 06:00 (Mon.),新竹縣,12小時降雨機率,PoP12h,10,%,120.995698,24.841245,10%,2023/08/06 (Sun.),新竹縣,北部地區
2,2023/08/07 06:00 (Mon.),2023/08/07 18:00 (Mon.),新竹縣,12小時降雨機率,PoP12h,10,%,120.995698,24.841245,10%,2023/08/07 (Mon.),新竹縣,北部地區
3,2023/08/07 18:00 (Mon.),2023/08/08 06:00 (Tue.),新竹縣,12小時降雨機率,PoP12h,10,%,120.995698,24.841245,10%,2023/08/07 (Mon.),新竹縣,北部地區
4,2023/08/08 06:00 (Tue.),2023/08/08 18:00 (Tue.),新竹縣,12小時降雨機率,PoP12h,10,%,120.995698,24.841245,10%,2023/08/08 (Tue.),新竹縣,北部地區
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5539,2023/08/10 18:00 (Thu.),2023/08/11 06:00 (Fri.),新竹市,平均露點溫度,Td,25,°C,120.962110,24.818109,25°C,2023/08/10 (Thu.),新竹市,北部地區
5540,2023/08/11 06:00 (Fri.),2023/08/11 18:00 (Fri.),新竹市,平均露點溫度,Td,27,°C,120.962110,24.818109,27°C,2023/08/11 (Fri.),新竹市,北部地區
5541,2023/08/11 18:00 (Fri.),2023/08/12 06:00 (Sat.),新竹市,平均露點溫度,Td,26,°C,120.962110,24.818109,26°C,2023/08/11 (Fri.),新竹市,北部地區
5542,2023/08/12 06:00 (Sat.),2023/08/12 18:00 (Sat.),新竹市,平均露點溫度,Td,24,°C,120.962110,24.818109,24°C,2023/08/12 (Sat.),新竹市,北部地區


In [74]:
data_df_pivot = pd.pivot_table(data_df,
                               index=["StartTime", "Date", "EndTime", "Region", 'Location'], 
                               columns=["Element"], 
                               values=["Parameter"],
                               aggfunc= lambda x: x).reset_index()
data_df_pivot.columns = ["StartTime","Date","EndTime", "Region","Location","12hr PoP", "Weather FCST", "Weather Desc", 
                         "AvgT", "AvgRH","AvgDPT","EL","MinT","MinAT","MaxC","MaxCI",
                         "MaxWS","MinC","MinCI","MaxT","MaxAT","UVI","BWS","WD"]
# 平均相對濕度、平均露點溫度、曝曬級數、蒲福風級
data_df_pivot = data_df_pivot.sort_values(by=["Region","Location","StartTime","EndTime"]).reset_index(drop=True)
data_df_pivot_graph = data_df_pivot.copy()[["StartTime","Location","MinT","MaxT","AvgT","AvgDPT","MinAT","MaxAT"]]
data_df_pivot["Period"] = data_df_pivot["StartTime"].str.cat(data_df_pivot["EndTime"], sep =" ~ ")
data_df_pivot["T"] = data_df_pivot["MinT"].str.cat(data_df_pivot["MaxT"], sep =" ~ ")
data_df_pivot["AT"] = data_df_pivot["MinAT"].str.cat(data_df_pivot["MaxAT"], sep =" ~ ")
data_df_pivot["C"] = data_df_pivot["MinC"].str.cat(data_df_pivot["MaxC"], sep =" ~ ")
data_df_pivot["CI"] = data_df_pivot["MinCI"].str.cat(data_df_pivot["MaxCI"], sep =" ~ ")
data_df_pivot["EL"].fillna("-", inplace=True)
data_df_pivot["CI"] = data_df_pivot["CI"]+" ("+data_df_pivot["C"]+")"
data_df_pivot["UVI"] = data_df_pivot["UVI"]+" ("+data_df_pivot["EL"]+")"
data_df_pivot["UVI"].fillna("-", inplace=True)
data_df_pivot = data_df_pivot[["Date","Period","Region","Location","Weather FCST","12hr PoP","T","AvgT","AT",
                               "AvgRH","AvgDPT","UVI",
                               "MaxWS", "BWS", "WD","CI", "Weather Desc"]]
data_df_pivot.head(23)
data_df_pivot[data_df_pivot["Location"]=="台北市"]
data_df_pivot_graph['MinT'] = data_df_pivot_graph['MinT'].apply(lambda x:int(x.replace("°C","")))
data_df_pivot_graph['MaxT'] = data_df_pivot_graph['MaxT'].apply(lambda x:int(x.replace("°C","")))
data_df_pivot_graph["AvgT"] = data_df_pivot_graph["AvgT"].apply(lambda x:int(x.replace("°C","")))
data_df_pivot_graph["AvgDPT"] = data_df_pivot_graph["AvgDPT"].apply(lambda x:int(x.replace("°C","")))
data_df_pivot_graph["MinAT"] = data_df_pivot_graph["MinAT"].apply(lambda x:int(x.replace("°C","")))
data_df_pivot_graph["MaxAT"] = data_df_pivot_graph["MaxAT"].apply(lambda x:int(x.replace("°C","")))
data_df_pivot_graph["StartTime"] = data_df_pivot_graph["StartTime"].apply(lambda x:datetime.strptime(x.split("(")[0].strip(), '%Y/%m/%d %H:%M'))
data_df_pivot_graph["StartTime"] = pd.to_datetime(data_df_pivot_graph['StartTime'])

data_df_pivot_graph[:10]

,StartTime,Location,MinT,MaxT,AvgT,AvgDPT,MinAT,MaxAT
0,2023-08-06 12:00:00,南投縣,29,32,31,28,35,39
1,2023-08-06 18:00:00,南投縣,26,29,27,25,30,35
2,2023-08-07 06:00:00,南投縣,26,33,30,26,30,39
3,2023-08-07 18:00:00,南投縣,26,30,27,26,31,34
4,2023-08-08 06:00:00,南投縣,26,33,30,27,31,39
5,2023-08-08 18:00:00,南投縣,26,30,27,26,31,35
6,2023-08-09 06:00:00,南投縣,26,32,30,27,31,39
7,2023-08-09 18:00:00,南投縣,26,29,27,26,30,34
8,2023-08-10 06:00:00,南投縣,26,32,30,27,30,38
9,2023-08-10 18:00:00,南投縣,26,29,27,26,30,35


In [31]:
# data_df_pivot.to_excel("data_df_pivot.xlsx", index=False)

In [75]:
data_df_pivot_graph.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308 entries, 0 to 307
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   StartTime  308 non-null    datetime64[ns]
 1   Location   308 non-null    object        
 2   MinT       308 non-null    int64         
 3   MaxT       308 non-null    int64         
 4   AvgT       308 non-null    int64         
 5   AvgDPT     308 non-null    int64         
 6   MinAT      308 non-null    int64         
 7   MaxAT      308 non-null    int64         
dtypes: datetime64[ns](1), int64(6), object(1)
memory usage: 19.4+ KB


In [71]:
data_df_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308 entries, 0 to 307
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Date          308 non-null    object
 1   Period        308 non-null    object
 2   Region        308 non-null    object
 3   Location      308 non-null    object
 4   Weather FCST  308 non-null    object
 5   12hr PoP      308 non-null    object
 6   T             308 non-null    object
 7   AvgT          308 non-null    object
 8   AT            308 non-null    object
 9   AvgRH         308 non-null    object
 10  AvgDPT        308 non-null    object
 11  UVI           308 non-null    object
 12  MaxWS         308 non-null    object
 13  BWS           308 non-null    object
 14  WD            308 non-null    object
 15  CI            308 non-null    object
 16  Weather Desc  308 non-null    object
dtypes: object(17)
memory usage: 41.0+ KB


In [36]:
data_df_pivot_graph = data_df_pivot.copy()
data_df_pivot_graph[data_df_pivot_graph["Location"]=="台北市"]

,Date,Period,Region,Location,Weather FCST,12hr PoP,T,AvgT,AT,AvgRH,AvgDPT,UVI,MaxWS,BWS,WD,CI,Weather Desc
70,2023/08/06 (Sun.),2023/08/06 12:00 (Sun.) ~ 2023/08/06 18:00 (Sun.),北部地區,台北市,晴時多雲,10%,32°C ~ 36°C,34°C,37°C ~ 41°C,64%,26°C,10 (過量級),2m/s,2,偏西風,29 ~ 31 (悶熱 ~ 易中暑),晴時多雲。降雨機率 10%。溫度攝氏32至36度。悶熱至易中暑。偏西風 風速2級(每秒2公尺...
71,2023/08/06 (Sun.),2023/08/06 18:00 (Sun.) ~ 2023/08/07 06:00 (Mon.),北部地區,台北市,晴時多雲,10%,28°C ~ 32°C,29°C,32°C ~ 37°C,77%,25°C,-,2m/s,≤1,偏西風,26 ~ 29 (舒適 ~ 悶熱),晴時多雲。降雨機率 10%。溫度攝氏28至32度。舒適至悶熱。偏西風 風速<= 1級(每秒2...
72,2023/08/07 (Mon.),2023/08/07 06:00 (Mon.) ~ 2023/08/07 18:00 (Mon.),北部地區,台北市,晴時多雲,10%,28°C ~ 36°C,33°C,32°C ~ 42°C,71%,27°C,11 (危險級),3m/s,2,偏西風,26 ~ 32 (舒適 ~ 易中暑),晴時多雲。降雨機率 10%。溫度攝氏28至36度。舒適至易中暑。偏西風 風速2級(每秒3公尺...
73,2023/08/07 (Mon.),2023/08/07 18:00 (Mon.) ~ 2023/08/08 06:00 (Tue.),北部地區,台北市,晴時多雲,10%,28°C ~ 32°C,29°C,32°C ~ 38°C,82%,26°C,-,2m/s,≤1,偏西風,26 ~ 30 (舒適 ~ 悶熱),晴時多雲。降雨機率 10%。溫度攝氏28至32度。舒適至悶熱。偏西風 風速<= 1級(每秒2...
74,2023/08/08 (Tue.),2023/08/08 06:00 (Tue.) ~ 2023/08/08 18:00 (Tue.),北部地區,台北市,晴時多雲,20%,28°C ~ 34°C,32°C,32°C ~ 41°C,81%,28°C,9 (過量級),3m/s,2,偏西風,26 ~ 32 (舒適 ~ 易中暑),晴時多雲。降雨機率 20%。溫度攝氏28至34度。舒適至易中暑。偏西風 風速2級(每秒3公尺...
75,2023/08/08 (Tue.),2023/08/08 18:00 (Tue.) ~ 2023/08/09 06:00 (Wed.),北部地區,台北市,晴時多雲,10%,27°C ~ 31°C,29°C,32°C ~ 37°C,83%,25°C,-,1m/s,≤1,偏西風,26 ~ 30 (舒適 ~ 悶熱),晴時多雲。降雨機率 10%。溫度攝氏27至31度。舒適至悶熱。偏西風 風速<= 1級(每秒1...
76,2023/08/09 (Wed.),2023/08/09 06:00 (Wed.) ~ 2023/08/09 18:00 (Wed.),北部地區,台北市,多雲時陰短暫陣雨,-,27°C ~ 33°C,31°C,32°C ~ 40°C,81%,27°C,7 (高量級),2m/s,≤1,東北風,26 ~ 31 (舒適 ~ 易中暑),多雲時陰短暫陣雨。溫度攝氏27至33度。舒適至易中暑。東北風 風速<= 1級(每秒2公尺)。...
77,2023/08/09 (Wed.),2023/08/09 18:00 (Wed.) ~ 2023/08/10 06:00 (Thu.),北部地區,台北市,晴時多雲,-,27°C ~ 30°C,28°C,31°C ~ 35°C,86%,26°C,-,1m/s,≤1,西北風,26 ~ 28 (舒適 ~ 悶熱),晴時多雲。溫度攝氏27至30度。舒適至悶熱。西北風 風速<= 1級(每秒1公尺)。相對濕度86%。
78,2023/08/10 (Thu.),2023/08/10 06:00 (Thu.) ~ 2023/08/10 18:00 (Thu.),北部地區,台北市,陰時多雲短暫陣雨,-,27°C ~ 32°C,30°C,31°C ~ 38°C,82%,27°C,7 (高量級),2m/s,≤1,偏東風,26 ~ 30 (舒適 ~ 悶熱),陰時多雲短暫陣雨。溫度攝氏27至32度。舒適至悶熱。偏東風 風速<= 1級(每秒2公尺)。相...
79,2023/08/10 (Thu.),2023/08/10 18:00 (Thu.) ~ 2023/08/11 06:00 (Fri.),北部地區,台北市,陰短暫陣雨,-,27°C ~ 29°C,28°C,31°C ~ 35°C,92%,27°C,-,2m/s,≤1,偏南風,26 ~ 28 (舒適 ~ 悶熱),陰短暫陣雨。溫度攝氏27至29度。舒適至悶熱。偏南風 風速<= 1級(每秒2公尺)。相對濕度...


In [ ]:
# print(set(data_df["Element"]))
# print(set(data_df["Element_EN"]))
# print(set(data_df["Unit"])) # Beaufort Wind Scale
# print(set(data_df["StartTime"]))